In [463]:
import pymorphy2
from multiprocessing.dummy import Pool, Queue
import lxml
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from bs4.element import Comment
import gzip
import pandas as pd
import codecs
import sys
import os
from urllib.parse import urlparse
import json
import tldextract
import re
from nltk.stem.snowball import SnowballStemmer

In [462]:
!pip install tldextract

In [446]:
regex_symbols = re.compile(r'[\n\r\t|()#№^«»↓@!_&/><?%\[\]©·•]')

In [467]:
for i in range(1):
    with open('./content/7.dat', encoding='utf-8') as file:
        text_checker = {}
        html_test = file.read()
        soup = BeautifulSoup(html_test, 'html.parser')
        title_name = soup.title.text
        url = soup.text[:soup.text.index('\n')]
        url = tldextract.extract(url)
        url = url.domain + '.' + url.suffix
        text = soup.text[soup.text.index('\n'):]
        text = regex_symbols.sub(" ", text)
        text = re.sub("\s\s+" , " ", text)
        text = re.sub('\xa0|\xad', ' ', text)
        title_name = regex_symbols.sub(" ", title_name)
        title_name = re.sub("\s\s+" , " ", title_name)     
        text_checker[1] = (url, title_name, text)
text_checker

{1: ('beon.ru',
  'Изюм',
  ' Изюм Привет, Гость Войти… Регистрация Сообщества Опросы Тесты Фоторедактор Интересы Поиск пользователей Дуэли Аватары Гороскоп Кто, Где, Когда Игры В онлайне Позитивки 2048 new Случайный дневник BeOn Ещё… вниз ЗарегистрироватьсяЛогин:Пароль:Запомнить меня Забыли пароль yesПолучи свой дневник Дневник пользователя vlad kakashi kubrak Изюм записи, возможно интересные автору дневника Реклама: Разместить рекламу здесь… О себе: привет я влад занимаюсь боевыми искуствами люблю слушать музыку и гулять с друзьями. Интересы: Боевые искуства наруто и девушки Мой альбом Моё Избранное и Цитатник Мои Дискуссии и Интервью Последние комментарии Записи и Занятия друзей Дни рождения друзей Знаки Зодиака друзей Мои ссылки и Изюм Мои желания и Дуэли ещё обо мне… Статус: offline — записи — комментарии — ленты друзей — избранное Реклама: Разместить рекламу здесь… Мои категории: Vip Мои друзья: Брусиидаl2009lА.М.подробнее Меня читают: 2311 дней назад Брусиида2435 дней назад на В

In [411]:
my_position_start = 1
my_position_end = 10
queue = Queue() # очередь ссылок на книги
for i in range(my_position_start, my_position_end):
    queue.put(i)

In [412]:
def process_doc(number_of_doc):
    with open('./content/{:d}.dat'.format(number_of_doc), encoding='utf-8') as file:
        text_checker = {}
        html_test = file.read()
        soup = BeautifulSoup(html_test, 'html.parser')
        print(soup.text)
        title_of_doc = soup.text[:soup.text.index('\n')]
        text = soup.text[soup.text.index('\n'):]
        text = regex_symbols.sub(" ", text)
        text = re.sub('title_of_doc', '', text)
        text = re.sub("\s\s+" , " ", text)
        text_checker[number_of_doc] = (title_of_doc, text) 
        text_checker[title_of_doc] = text
        return text_checker

In [410]:
def process_all_docs(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            try:
                id_new = queue.get()
                record = process_doc(id_new)
            except Exception as e:
                print(id_new, file=sys.stderr)
                print(e, file=sys.stderr)
            record_str = json.dumps(record, ensure_ascii=False)
            print(record_str, file=f_json)

            # счетчик должен атомарно обновиться
            with lock:
                pbar.update(1)


with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_all_docs, range(pool._processes))

 11%|█         | 1/9 [00:00<00:01,  5.68it/s]

1
23
45
6


7
8


 22%|██▏       | 2/9 [00:00<00:01,  4.08it/s]

9


100%|██████████| 9/9 [00:01<00:00,  7.72it/s]
